In [1]:
# !pip install chromadb sentence_transformers

In [2]:
import chromadb
from sentence_transformers import SentenceTransformer
import torch

# ---------------------------------------------------------
# 1. 설정 (GPU 확인 등)
# ---------------------------------------------------------
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# ---------------------------------------------------------
# 2. 모델 로드 (저장할 때 썼던 그 모델!)
# ---------------------------------------------------------
print("모델 로드 중...")
model = SentenceTransformer("dragonkue/BGE-m3-ko").to(device)

# ---------------------------------------------------------
# 3. ChromaDB 연결 (다운로드 받은 폴더 경로 지정)
# ---------------------------------------------------------
# path="./chroma_db" 는 압축 푼 폴더 이름과 같아야 합니다.
client = chromadb.PersistentClient(path="./chroma_db")

# 컬렉션 가져오기 (create가 아니라 get_collection 사용)
collection = client.get_collection(name="patent_claims")

print(f"✅ 데이터베이스 로드 완료! 총 데이터 수: {collection.count()}개")



c:\Users\kouls\anaconda3\envs\llm_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cpu
모델 로드 중...
✅ 데이터베이스 로드 완료! 총 데이터 수: 589049개


In [3]:
from collections import defaultdict
import math

### 1차 검색

### 특허 단위 점수 집계(Late Fusion Aggregated Scoring)
- 출원번호 하나당 모든 ‘매칭된 청구항’들의 유사도 점수를 통계적으로 합성해서 특허 단위 점수를 만듦
- 단점: 독립항이 핵심인데 종속항이 우연히 많이 매칭되면 점수가 잘못 올라갈 수 있음

In [4]:
def many_claim_dis(results, TOP_K):
    # ----------------------------------------
    # 0. Chroma 결과 파싱
    # ----------------------------------------
    ids        = results["ids"][0]
    docs       = results["documents"][0]
    metas      = results["metadatas"][0]
    distances  = results["distances"][0]

    parsed = []
    for i in range(len(ids)):
        parsed.append({
            "id": ids[i],
            "document": docs[i],
            "metadata": metas[i],
            "distance": distances[i]
        })

    # ----------------------------------------
    # 1. 출원번호 기준 그룹화
    # ----------------------------------------
    grouped = defaultdict(list)
    for r in parsed:
        app_no = r["metadata"]["patent_id"]
        grouped[app_no].append(r)

    # ----------------------------------------
    # 2. 특허 단위 점수 계산
    #    방법: claim similarity들의 평균 + 대표 claim 보정
    # ----------------------------------------

    def similarity(distance):
        return 1.0 / (1.0 + distance)

    def compute_patent_score(claims):
        """
        claims : 특정 출원번호의 claim 리스트
        점수 구성:
        - avg_similarity  : 전체 claim 유사도의 평균
        - max_similarity  : 가장 유사한 claim의 similarity
        - claim_count_penalty: claim 개수가 많을수록 약간 가산
        """
        sims = [similarity(c["distance"]) for c in claims]
        avg_sim = sum(sims) / len(sims)
        max_sim = max(sims)
        
        # 보조 요인: 클레임이 여러 개 반환됐다면 관련성이 높다고 가정
        count_bonus = min(1.0, len(claims) / 8.0)  # 8개 이상이면 보너스 최대치
        
        # 최종 점수 조합
        final_score = avg_sim * 0.7 + max_sim * 0.3 + count_bonus * 0.05
        return final_score

    # ----------------------------------------
    # 3. 특허 단위 재랭킹
    # ----------------------------------------
    aggregated = []
    for app_no, claims in grouped.items():
        score = compute_patent_score(claims)

        # 대표 claim은 거리(distance)가 가장 낮은 claim 선택
        rep_claim = sorted(claims, key=lambda x: x["distance"])[0]

        aggregated.append({
            "patent_id": app_no,
            "score": score,
            "origin_claim": rep_claim["document"],
            "claim_no": rep_claim["metadata"]["claim_no"],
            "claims_found": len(claims),
            "claims": claims
        })

    # 점수 높은 순으로 재랭킹
    aggregated = sorted(aggregated, key=lambda x: x["score"], reverse=True)

    final_response = aggregated[:TOP_K]
    return final_response

### 평가 함수

In [5]:
def eval(results, ipc_list, TOP_K):
    final_response = many_claim_dis(results, TOP_K)

    # 비교
    response_patent_id = [f['patent_id'] for f in final_response]

    print(ipc_list & set(response_patent_id))

    print("Precision:", len(ipc_list & set(response_patent_id)) / TOP_K)
    print("Recall:", len(ipc_list & set(response_patent_id)) / len(ipc_list))

### query가 2개 이상일 때 re-ranking해서 200건만 저장하는 함수

In [6]:
# 200개 초과 검색되었을 때 re-ranking
import numpy as np

def multi_query_rerank(
    collection,
    model,
    query_list,
    per_query_top_k=200,
    final_top_k=200
):
    #------------------------------------------
    # 1) Q개의 query 문장 embedding
    #------------------------------------------
    query_embs = model.encode(query_list).tolist()

    candidates = []  # 전체 후보 저장
    
    #------------------------------------------
    # 2) query별 independent 검색
    #------------------------------------------
    for emb in query_embs:
        r = collection.query(
            query_embeddings=[emb],
            n_results=per_query_top_k
        )

        ids = r["ids"][0]
        docs = r["documents"][0]
        distances = r["distances"][0]
        metas = r["metadatas"][0]

        # 후보를 통합 리스트에 추가
        for pid, doc, dist, meta in zip(ids, docs, distances, metas):
            candidates.append({
                "id": pid,
                "document": doc,
                "distance": dist,
                'metadatas':meta
            })

    #------------------------------------------
    # 3) distance 기준 정렬 (오름차순)
    #------------------------------------------
    candidates = sorted(candidates, key=lambda x: x["distance"])

    #------------------------------------------
    # 4) 상위 final_top_k만 선택
    #------------------------------------------
    top_candidates = candidates[:final_top_k]

    #------------------------------------------
    # 5) collection.query() 형식으로 재구성
    #------------------------------------------
    final_ids = [c["id"] for c in top_candidates]
    final_docs = [c["document"] for c in top_candidates]
    final_distances = [c["distance"] for c in top_candidates]
    final_metas = [c['metadatas'] for c in top_candidates]

    final_results = {
        "ids": [final_ids],
        "documents": [final_docs],
        "distances": [final_distances],
        "metadatas": [final_metas]
    }

    return final_results

---

In [7]:
# 정답 데이터 로드
import pandas as pd

data = pd.read_csv('./test_data.csv')
patent_ids = set(data['출원번호'].astype(str))  

# 확인
print(f"총 {len(patent_ids)}개의 출원번호")
print("샘플:", list(patent_ids)[:5])

총 500개의 출원번호
샘플: ['1020257024391', '1020157021945', '1020240078785', '1020137014291', '1020220139807']


In [9]:
# 실제 반환되는 patent_id 확인
query = [
    "영상 처리 3D Mapping 시스템 소프트웨어 데이터 처리 MCU 제어부", 
    "컴퓨터 입력 장치 키보드 터치판 사용자 인터페이스 소프트웨어 시스템"
]

results = multi_query_rerank(
    collection=collection,
    model=model,
    query_list=query,
    per_query_top_k=200,
    final_top_k=200
)

# many_claim_dis 함수로 처리
final_response = many_claim_dis(results, TOP_K=10)

# 실제 patent_id 출력
print("=== 실제 반환된 patent_id 샘플 ===")
for i, r in enumerate(final_response[:10]):
    print(f"{i+1}. patent_id: '{r['patent_id']}' (type: {type(r['patent_id'])})")

# test_data의 patent_id 샘플 출력
print("\n=== test_data.csv의 patent_id 샘플 ===")
for i, pid in enumerate(list(patent_ids)[:10]):
    print(f"{i+1}. patent_id: '{pid}' (type: {type(pid)})")


=== 실제 반환된 patent_id 샘플 ===
1. patent_id: '1020217039153' (type: <class 'str'>)
2. patent_id: '1020220113402' (type: <class 'str'>)
3. patent_id: '1020257003112' (type: <class 'str'>)
4. patent_id: '1020207026278' (type: <class 'str'>)
5. patent_id: '1020190106521' (type: <class 'str'>)
6. patent_id: '1020247014679' (type: <class 'str'>)
7. patent_id: '1020257006968' (type: <class 'str'>)
8. patent_id: '1020230166856' (type: <class 'str'>)
9. patent_id: '1020257001491' (type: <class 'str'>)
10. patent_id: '1020257010889' (type: <class 'str'>)

=== test_data.csv의 patent_id 샘플 ===
1. patent_id: '1020257024391' (type: <class 'str'>)
2. patent_id: '1020157021945' (type: <class 'str'>)
3. patent_id: '1020240078785' (type: <class 'str'>)
4. patent_id: '1020137014291' (type: <class 'str'>)
5. patent_id: '1020220139807' (type: <class 'str'>)
6. patent_id: '1020187029464' (type: <class 'str'>)
7. patent_id: '1020117029455' (type: <class 'str'>)
8. patent_id: '1020220101793' (type: <class 'str'>

In [11]:
query = [
    "컴퓨터 비전 영상 분석 2D 3D Mapping 좌표 기록",
    "산업 안전 CCTV 자세 감지 경고 시스템"
]

results = multi_query_rerank(
    collection=collection,
    model=model,
    query_list=query,
    per_query_top_k=200,
    final_top_k=200
)

TOP_K = 30
eval(results, patent_ids, TOP_K)

{'1020230155140'}
Precision: 0.03333333333333333
Recall: 0.002
